In [1]:
# #Mount your drive
# from google.colab import drive
# drive.mount('/content/drive')

# Code
1.   Implement and apply kNN on MNIST with k=1, 5, 10. Apply kNN on raw images, and 2, 7 dimensional eigenspaces, respectively. Show the accuracy scores for each run (you'd run the algorithm 9 times).

2.   Use and run the Random Forest algorithm for MNIST classification (http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html). Show the accuracy scores.

If it takes too much time, reduce the number of samples for training/testing (based on random selection).

In [2]:
#Code here
# Code here
import six.moves.cPickle as pickle
import gzip
import os
import numpy as np
from PIL import Image
from numpy import linalg as LA
import matplotlib.pyplot as plt
import urllib.request
from numpy.random import uniform
from sklearn.decomposition import PCA

In [3]:
def load_data(dataset):
    ''' Loads the dataset

    :type dataset: string
    :param dataset: the path to the dataset (here MNIST)

    copied from http://deeplearning.net/ and revised by hchoi
    '''

    # Download the MNIST dataset if it is not present
    if not os.path.isfile(dataset):
        origin = (
            'http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz'
        )
        print('Downloading data from %s' % origin)
        urllib.request.urlretrieve(origin, dataset)

    print('Loading data...')

    # Load the dataset
    with gzip.open(dataset, 'rb') as f:
        try:
            train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
        except:
            train_set, valid_set, test_set = pickle.load(f)
    # train_set, valid_set, test_set format: tuple(input, target)
    # input is a numpy.ndarray of 2 dimensions (a matrix)
    # where each row corresponds to an example. target is a
    # numpy.ndarray of 1 dimension (vector) that has the same length as
    # the number of rows in the input. It should give the target
    # to the example with the same index in the input.
    print('... data has been loaded!')
    return train_set, valid_set, test_set

In [4]:
#Load the data into train, validation and test sets
train_set, val_set, test_set = load_data('mnist.pkl.gz')

#Separate each set into image vector (_x) and label (_y)
train_x, train_y = train_set
val_x, val_y = val_set
test_x, test_y = test_set

desired_size = 5000

#Due to Computational Expensiveness

np.random.seed(0)
indices = np.random.choice(train_x.shape[0], desired_size, replace=False)
train_x = train_x[indices]
train_y = train_y[indices]

print(train_x.shape)
print(train_y.shape)

desired_size = 5000

np.random.seed(0)
indices = np.random.choice(test_x.shape[0], desired_size, replace=False)
test_x = test_x[indices]
test_y = test_y[indices]

print(test_x.shape)
print(test_y.shape)

Loading data...
... data has been loaded!
(5000, 784)
(5000,)
(5000, 784)
(5000,)


In [5]:
def predictNeighbors(train_x_set, train_y_set, test_x, k):
  dists = []
  for i, train_row in enumerate(train_x_set):
      dist = np.linalg.norm(train_row - test_x)
      dists.append((train_y_set[i], dist))
  dists.sort(key=lambda x: x[1])

  neighbors = dists[:k]

  class_counts = {}
  for neighbor in neighbors:
      label, _ = neighbor
      if label in class_counts:
          class_counts[label] += 1
      else:
          class_counts[label] = 1

  predicted_class = max(class_counts, key=class_counts.get)

  return predicted_class

In [6]:
def calculate_accuracy(train_x_set, train_y, test_x_set, test_y, k):

    correct_predictions = 0
    total_predictions = len(test_x_set)

    for i, test_x in enumerate(test_x_set):

        predicted_class = predictNeighbors(train_x_set, train_y, test_x, k)
        if predicted_class == test_y[i]:
            correct_predictions += 1

    accuracy = correct_predictions / total_predictions
    return accuracy

In [13]:
accuracy = calculate_accuracy(train_x, train_y, test_x, test_y, k=1)
print(f"Accuracy: {accuracy * 100:.2f}%")

accuracy = calculate_accuracy(train_x, train_y, test_x, test_y, k=5)
print(f"Accuracy: {accuracy * 100:.2f}%")

accuracy = calculate_accuracy(train_x, train_y, test_x, test_y, k=10)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 94.08%
Accuracy: 94.40%
Accuracy: 93.92%


In [8]:
#code here
pca = PCA(n_components=2)
PCA_2_train_x = pca.fit_transform(train_x)
PCA_2_test_x = pca.transform(test_x)
pca = PCA(n_components=7)
PCA_7_train_x = pca.fit_transform(train_x)
PCA_7_test_x = pca.transform(test_x)

In [9]:
accuracy = calculate_accuracy(PCA_2_train_x, train_y, PCA_2_test_x, test_y, k=1)
print(f"Accuracy: {accuracy * 100:.2f}%")

accuracy = calculate_accuracy(PCA_2_train_x, train_y, PCA_2_test_x, test_y, k=5)
print(f"Accuracy: {accuracy * 100:.2f}%")

accuracy = calculate_accuracy(PCA_2_train_x, train_y, PCA_2_test_x, test_y, k=10)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 37.90%
Accuracy: 42.14%
Accuracy: 44.16%


In [10]:
accuracy = calculate_accuracy(PCA_7_train_x, train_y, PCA_7_test_x, test_y, k=1)
print(f"Accuracy: {accuracy * 100:.2f}%")

accuracy = calculate_accuracy(PCA_7_train_x, train_y, PCA_7_test_x, test_y, k=5)
print(f"Accuracy: {accuracy * 100:.2f}%")

accuracy = calculate_accuracy(PCA_7_train_x, train_y, PCA_7_test_x, test_y, k=10)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 81.22%
Accuracy: 84.96%
Accuracy: 86.06%


Random Forrest

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


rf=RandomForestClassifier(n_estimators=100)
rf.fit(train_x,train_y)

y_predict = rf.predict(test_x)
print(f"Accuracy is {accuracy_score(y_predict, test_y)* 100} %")

Accuracy is 94.08 %


# ChatGPT prompts

Now solve the same task with ChatGPT with the following prompts:


1.   Can you implement and apply kNN on MNIST (from scratch without using any specialized library) with k=1, 5, 10? Can you show the accuracies of the predictions? Ignore the computational cost of this code.
2.   Can you use the same provided knn algorithm and apply it to the MNIST data with reduced dimensions to 2 and 7, with k=1, 5, 10? Please provide the accuracies.
3.   Can you provide a code for the random forest algorithm applied to the MNIST dataset? Please use specialized libraries like sklearn. Please show the accuracies scores.


**Paste the answer provided and run the code below:**

In [12]:
import numpy as np
from collections import Counter

# Function to calculate Euclidean distance between two images
def euclidean_distance(image1, image2):
    return np.sqrt(np.sum((image1 - image2) ** 2))

# Function to predict the label of a test image using k-NN
def predict_knn(train_data, train_labels, test_image, k):
    distances = []
    for i in range(len(train_data)):
        dist = euclidean_distance(train_data[i], test_image)
        distances.append((train_labels[i], dist))

    distances.sort(key=lambda x: x[1])
    k_nearest = [label for label, _ in distances[:k]]
    most_common = Counter(k_nearest).most_common(1)
    return most_common[0][0]

# Function to calculate accuracy
def calculate_accuracy(train_data, train_labels, test_data, test_labels, k):
    correct = 0
    for i in range(len(test_data)):
        prediction = predict_knn(train_data, train_labels, test_data[i], k)
        if prediction == test_labels[i]:
            correct += 1
    accuracy = correct / len(test_data)
    return accuracy

# Load MNIST dataset (you need to have the data in the appropriate format)
# Replace 'train_data', 'train_labels', 'test_data', and 'test_labels' with your data
# Each image in 'train_data' and 'test_data' should be a flattened array of pixel values, and labels should be integers.
# Ensure that train_data and test_data have the same number of features (784 for MNIST).


# Comments: ChatGPT kept failed to load MNIST dataset, So I provided context
train_data = train_x
train_labels = train_y
test_data = test_x
test_labels = test_y

# Set k values
k_values = [1, 5, 10]

# Calculate and print accuracies for each k
for k in k_values:
    accuracy = calculate_accuracy(train_data, train_labels, test_data, test_labels, k)
    print(f'Accuracy for k={k}: {accuracy}')

# Reduce dimensionality to 2 and 7 using PCA for both training and testing data
pca_2 = PCA(n_components=2)
reduced_train_data_2 = pca_2.fit_transform(train_data)
reduced_test_data_2 = pca_2.transform(test_data)
pca_7 = PCA(n_components=7)
reduced_train_data_7 = pca_7.fit_transform(train_data)
reduced_test_data_7 = pca_7.transform(test_data)

# Calculate and print accuracies for dimensionality reduction to 2
print("Accuracy for Dimensionality Reduction to 2:")
for k in k_values:
    accuracy = calculate_accuracy(reduced_train_data_2, train_labels, reduced_test_data_2, test_labels, k)
    print(f'Accuracy for k={k}: {accuracy}')

# Calculate and print accuracies for dimensionality reduction to 7
print("\nAccuracy for Dimensionality Reduction to 7:")
for k in k_values:
    accuracy = calculate_accuracy(reduced_train_data_7, train_labels, reduced_test_data_7, test_labels, k)
    print(f'Accuracy for k={k}: {accuracy}')

import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load the MNIST dataset (you need to have the data in the appropriate format)
# Replace 'X' with your data (each row is a flattened image), and 'y' with your labels.
# Ensure that X has the same number of features (784 for MNIST).

X = train_x  # Replace with your feature data
y = train_y  # Replace with your labels

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the classifier to the training data
rf_classifier.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = rf_classifier.predict(X_test)

# Calculate accuracy score
accuracy = accuracy_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')

Accuracy for k=1: 0.9408
Accuracy for k=5: 0.944
Accuracy for k=10: 0.9392
Accuracy for Dimensionality Reduction to 2:
Accuracy for k=1: 0.3794
Accuracy for k=5: 0.4212
Accuracy for k=10: 0.4416

Accuracy for Dimensionality Reduction to 7:
Accuracy for k=1: 0.8122
Accuracy for k=5: 0.8494
Accuracy for k=10: 0.8604
Accuracy: 0.925


# Report


1.   What are some differences you notice between your code and ChatGPT's? (Name at least 3)
2.   Ask ChatGPT to compare kNN and Random Forest. Paste the answer here. Do you agree with ChatGPT's statement? Why yes or why not?
3.   In yours and ChatGPTs implementation, which algorithm had the highest accuracy?
4.   Conclude with some thoughts and things you learned from this homework.

What are some differences you notice between your code and ChatGPT's? (Name at least 3)
1.	ChatGPT defined its own function to calculate Euclidean distance, but I used a built-in function in NumPy which is np.linalg.norm function
2.	To count the neighbors, I sorted the list by the shortest ones and counted each label not using any libraries. ChatGPT’s code used “from collections import Counter” to count the most common labels in the list.
3.	For the random forest implementation, ChatGPT used train_x and train_y for the whole data and label set and split into train and test dataset again. However, my code used train and data set as I used. I think it is ChatGPT’s limitation, just providing higher(frequent) tokens.


Ask ChatGPT to compare kNN and Random Forest. Paste the answer here. Do you agree with ChatGPT's statement? Why yes or why not?

k-Nearest Neighbors (k-NN) and Random Forest are both machine learning algorithms used for classification tasks, but they have different characteristics, advantages, and disadvantages. Let's compare these two algorithms:

1. Model Type:

    k-NN: k-NN is an instance-based or lazy learning algorithm. It makes predictions based on the majority class among the k-nearest neighbors of a data point.
    Random Forest: Random Forest is an ensemble learning method that combines multiple decision trees to make predictions.

2. Training:

    k-NN: k-NN doesn't involve traditional training. It stores the entire training dataset in memory and uses it for making predictions.
    Random Forest: Random Forest trains multiple decision trees on different subsets of the data with bootstrapping (bagging) and feature selection (random feature subspace). Each tree is trained independently.

3. Interpretability:

    k-NN: It is straightforward to understand and explain since it's based on the similarity of data points.
    Random Forest: It's more complex and challenging to interpret because it consists of many decision trees.

4. Complexity:

    k-NN: Predictions can be slow, especially for large datasets, as it needs to calculate distances to all training points.
    Random Forest: Predictions are generally faster due to the ensemble of decision trees. Training can be slower, but it can be parallelized.

5. Overfitting:

    k-NN: Prone to overfitting, especially with small values of k. The choice of k is crucial.
    Random Forest: Less prone to overfitting, thanks to the ensemble nature and the randomness introduced during training.

6. Parameter Selection:

    k-NN: Main parameter is k (the number of neighbors to consider). You need to choose an appropriate k value.
    Random Forest: Parameters include the number of trees, maximum depth of trees, and others. Hyperparameter tuning is required.

7. Performance:

    k-NN: It may perform well with small to medium-sized datasets, but it can suffer from the "curse of dimensionality" as the number of features increases.
    Random Forest: Generally performs well on a wide range of datasets, including high-dimensional data.

8. Handling Imbalanced Data:

    k-NN: Sensitive to imbalanced datasets as it may be biased toward the majority class.
    Random Forest: Better at handling imbalanced data, as it considers class balances during tree construction.

9. Versatility:

    k-NN: Suitable for a wide range of applications but less versatile than Random Forest.
    Random Forest: Versatile and applicable to various machine-learning tasks, including classification and regression.

10. Scalability:
- k-NN: Scales poorly with many data points due to the need for calculating distances to all training samples.
- Random Forest: More scalable and can handle large datasets.

In summary, the choice between k-NN and Random Forest depends on the specific characteristics of your dataset, the desired interpretability, and the trade-offs between training and prediction time. Random Forest is often a robust choice for many scenarios, while k-NN can be useful for smaller datasets with low dimensionality and simple interpretability requirements.

My answer: I agreed with ChatGPT’s answer. ChatGPT answered the difference between kNN and Random Forest in the various perspectives. It is true that k-NN is instance-based and lazy learning, and Random Forest is ensemble learning. Also, k-NN is straightforward Random Forest is more complex. Random Forest is ensemble learning combining multiple decision trees. The decision tree is quite straightforward, but its ensemble learning is more complicated However, I think both algorithms are quite straightforward other than machine learning algorithms. I also agreed with other opinions as well. Most importantly, k-NN takes a long for the large dataset since it requires the calculation of all datasets. Also, Random Forest can be slower, but it might be better to use it on a large dataset since the prediction is faster.

In yours and ChatGPTs implementation, which algorithm had the highest accuracy?

The highest accuracy algorithm is kNN algorithm with 5 neighbors However, there is no big difference between the random forest. It might be different depending on the dataset and number of datasets.

Conclude with some thoughts and things you learned from this homework.

In this homework, I could see how kNN works. kNN algorithms are popular algorithms that are used in various situations. Also, we conducted kNN on the MNIST dataset with different dimensions. Using 2-dimensional eigenspaces, the accuracy has been decreased significantly. Only reducing the dimensions is not the best choice all the time. Also, it was the first time to see how ensemble learning works. Using the sets of the model and choose can be the breakthrough.
